In [ ]:
from google.colab import drive # connect to google drive to load the model from google drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
# loading in the model from google drive (path varies based on user)

from tensorflow import keras
model_path = "/content/drive/MyDrive/UU/Pattern Recognition/PR project/1. Model training/final_model.h5" # depends on YOUR google drive! change accordingly
model = keras.models.load_model(model_path)
model.summary()



# image to test stuff on
PATH_TO_IMAGES = "/content/drive/MyDrive/UU/Pattern Recognition/PR project/3. Experimental setup/16 resized posters (300x400)"
img_path = "/content/drive/MyDrive/UU/Pattern Recognition/PR project/3. Experimental setup/16 resized posters (300x400)/6912 (action).jpg" # example image

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, None, None, None)  0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 2,260,546
Trainable params: 414,722
Non-tr

# Make prediction

In [ ]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
from google.colab import files

def make_prediction(im_path):
  image = Image.open(im_path)
  new_image = image.resize((224, 224))
  plt.imshow(new_image)
  new_image = np.asarray(new_image).reshape(1, 224, 224, 3)

  prediction = model.predict(new_image)
  print("Model output:", prediction, "\nClass prediction:", np.argmax(prediction))

make_prediction(img_path) #this is just an image I uploaded to the colab

# SHAP

Closely following this tutorial: https://medium.com/@tibastar/explain-the-prediction-for-imagenet-using-shap-468ec5bc9904

In [ ]:
! pip install shap

In [ ]:
import keras
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
import requests
from skimage.segmentation import slic
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import shap
import warnings

In [ ]:
# FUNCTION DEFINITIONS

from PIL import Image

def img_to_batch(img_path):
  # turn image into (1, 224, 224, 3), a single batch
  img = Image.open(img_path).resize((224, 224))
  # array representation of original image
  img_array = np.asarray(img)
  img_array.resize(1, 224, 224, 3)

  return img_array

def mask_image(zs, segmentation, image, background=None):
    if background is None:
        background = image.mean((0,1))
    out = np.zeros((zs.shape[0], image.shape[0], image.shape[1], image.shape[2]))
    for i in range(zs.shape[0]):
        out[i,:,:,:] = image
        for j in range(zs.shape[1]):
            if zs[i,j] == 0:
                out[i][segmentation == j,:] = background
    return out

# each z is basically a vector representing which segments are (1) or aren't (0)
# part of the current image being predicted on

# the function below takes a list of z vectors, passes that to mask image
# then mask_image generates a series of predictions corresponding 
# to EACH z vector, coming back with a group of predictions

def f(z): 
  # this becomes our wrapper model which can predict on a set of z vectors
  # the z vector itself is taken as a feature vector it seems (?)
    return model2.predict(mask_image(z, segments_slic, img_orig_array, 255))

def fill_segmentation(values, segmentation):
    out = np.zeros(segmentation.shape)
    for i in range(len(values)):
        out[segmentation == i] = values[i] # whichever pixels have index i get its value
    return out




## Define a function that generates SHAP explanation for file path of image

It returns TWO different 224x224 activation maps: one for '0' and one for '1' labels

In [ ]:
def predict_and_generate_shap_maps(img_path, return_prediction = False, verbose = True):

  # make a color map
  from matplotlib.colors import LinearSegmentedColormap
  colors = []
  for l in np.linspace(1,0,100):
      colors.append((245/255,39/255,87/255,l))
  for l in np.linspace(0,1,100):
      colors.append((24/255,196/255,93/255,l))
      
  cm = LinearSegmentedColormap.from_list("shap", colors)


  # open the image
  img = Image.open(img_path).resize((224, 224))

  # array representation of original image
  img_orig_array_local = np.asarray(img)

  # segment the image into 50 superpixels
  segments_slic_local = slic(img, n_segments=50, compactness=30, sigma=3)
  plt.imshow(segments_slic_local);
  plt.axis('off');

  # define predict function locally (for this image specifically)
  def f(z): 
  # this becomes our wrapper model which can predict on a set of z vectors
  # the z vector itself is taken as a feature vector it seems (?)
    return model.predict(mask_image(z, segments_slic_local, img_orig_array_local, 255))

  # use Kernel SHAP to explain the network's predictions under varying arrangements of masking
  explainer = shap.KernelExplainer(f, np.zeros((1,50)))
  with warnings.catch_warnings():
      warnings.simplefilter("ignore")
      shap_values = explainer.shap_values(np.ones((1,50)), nsamples=1000)

  # make prediction on the actual image
  preds = model.predict(img_orig_array_local.reshape(1,224,224,3))


  if verbose:
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12,4))
    inds = preds[0]
    axes[0].imshow(img)
    axes[0].axis('off')

  # max shap value
  max_val = np.max([np.max(np.abs(shap_values[i][:,:-1])) for i in range(len(shap_values))])


  # define dict of activation maps
  activation_maps_dict = {}



  for i in range(2): # we plot a shap map for each possible label: 0 and 1
      m = fill_segmentation(shap_values[i][0], segments_slic_local) # set segmenetation areas equal to shap values

      activation_maps_dict[str(i)] = m # save activation map to activation_maps dictionary

      if verbose:
        print(m.shape)
        axes[i+1].set_title(str(i))
        axes[i+1].imshow(img.convert('LA'), alpha=0.3)
        im = axes[i+1].imshow(m, cmap=cm, vmin=-max_val, vmax=max_val)
        axes[i+1].axis('off')
  if verbose:
    cb = fig.colorbar(im, ax=axes.ravel().tolist(), label="SHAP value", orientation="horizontal", aspect=60)
    cb.outline.set_visible(False)
    plt.show()

  if return_prediction == True:
    return activation_maps_dict, preds
  if return_prediction == False:
    return activation_maps_dict
    
# test!
# predict_and_generate_shap_maps(img_path)



Note that the previous code created a dictionary **activation_maps**

This dictionary holds two activation maps, for class '0' and class '1'

The activation maps are 224x224, they can be directly compared to the user's input, assuming we also scale it to 224x224

We have to discuss whether we will only consider positive values and set negative values to 0. This may make sense

In [ ]:
#activation_maps['1'].shape

# SmoothGrad

From https://github.com/sicara/tf-explain/tree/master/examples/core

In [ ]:
!pip install tf-explain

In [ ]:
from tf_explain.core.smoothgrad import SmoothGrad

def generate_smoothgrad_maps(img_path, class_indexes = [0,1]):
  grids = [] # it will return these grids

  img_batch = img_to_batch(img_path)

  explainer = SmoothGrad()

  for class_index in class_indexes:

    data = img_batch, class_index
    # Compute SmoothGrad on model
    grid = explainer.explain(data, model, class_index, 40, 2.0)

    grids.append(grid)
  
  return grids # return a grid for each target class index

# Occlusion

Documentation: https://github.com/sicara/tf-explain/blob/9d7d1e900ec3e3e4b5338fbc43dfb93539acecc2/tf_explain/core/occlusion_sensitivity.py#L14

Explanation from MatLab website: "The occlusionSensitivity function perturbs small areas of the input by replacing it with an occluding mask, typically a gray square. The mask moves across the image, and the change in probability score for a given class is measured as a function of mask position. You can use this method to highlight which parts of the image are most important to the classification: when that part of the image is occluded, the probability score for the predicted class will fall sharply."


In [ ]:
!pip install tf-explain

In [ ]:
from tf_explain.core.occlusion_sensitivity import OcclusionSensitivity

IMAGE_PATH = img_path

def generate_occlusion_maps(img_path, class_indexes = [0,1]):
  grids = [] # it will return these grids

  img_batch = img_to_batch(img_path)

  explainer = OcclusionSensitivity()

  for class_index in class_indexes:

    data = img_batch, class_index
    # Compute Occlusion Sensitivity for a given patch_size
    grid = explainer.get_sensitivity_map(model, data[0][0], class_index, 20)

    grids.append(grid)
  
  return grids # return a grid for each target class index


# Generate SHAP and GradCam maps for each image, save dataframe...

**Just in case, we also save the attention maps for the "practice" image**

In [ ]:
# get list of image files to generate attention maps for


import os
import re

all_files = os.listdir(PATH_TO_IMAGES)
image_files = []

for filename in all_files:
  if "jpg" in filename:
    image_files.append(filename)

maps_dict = {"id":[], "filename":[], "prediction":[], "true":[], "shap_0":[], "shap_1":[], "occlusion_0":[], "occlusion_1":[]}


# start filling dictionary with data for each file, including prediction, true value, and activation maps by methodn
# activation maps are stored as numpy arrays
for filename in image_files:
  file_path = os.path.join(PATH_TO_IMAGES, filename)

  if "action" in filename:
    id = filename.split("(")[0] #the id
    true_val = 1 # if filename contains "action", true_val = 1, otherwise true_val = 0
  else:
    id = filename.split(".")[0] # id is nr before dot
    true_val = 0

  # generate prediction, and also SHAP attention maps for each possible class label
  activation_map_dict, preds = predict_and_generate_shap_maps(file_path, return_prediction = True, verbose = False)
  shap_0 = activation_map_dict['0']
  shap_1 = activation_map_dict['1']

  occlusion_0, occlusion_1 = generate_occlusion_maps(file_path, class_indexes = [0,1])

  # add to dictionary
  maps_dict['id'].append(id)
  maps_dict['filename'].append(filename)
  maps_dict['prediction'].append(preds)
  maps_dict['true'].append(true_val)
  maps_dict['shap_0'].append(shap_0)
  maps_dict['shap_1'].append(shap_1)
  maps_dict['occlusion_0'].append(occlusion_0) # not yet implemented
  maps_dict['occlusion_1'].append(occlusion_1)

# create dataframe from dictionary
maps_df = pd.DataFrame.from_dict(maps_dict)
#maps_df.to_csv('newFile.csv', index = False)
    

In [ ]:
maps_df

# Graph all attention maps in dataframe

In [ ]:
nr_imgs = len(maps_df)
fig, axes = plt.subplots(nrows=nr_imgs, ncols=5, figsize = (12,nr_imgs*4));
#fig.tight_layout()

axes[0, 1].set_title("SHAP\n(not action)")

axes[0, 2].set_title("SHAP\n(action)")

axes[0, 3].set_title("Occlusion\n(not action)")

axes[0, 4].set_title("Occlusion\n(action)")

# make a color map
from matplotlib.colors import LinearSegmentedColormap
colors = []
for l in np.linspace(1,0,100):
    colors.append((245/255,39/255,87/255,l))
for l in np.linspace(0,1,100):
    colors.append((24/255,196/255,93/255,l))
    
cm = LinearSegmentedColormap.from_list("shap", colors)


for index, row in maps_df.iterrows():
  img_array = img_to_batch(os.path.join(PATH_TO_IMAGES, row['filename']))[0]
  axes[index, 0].imshow(img_array)
  axes[index, 0].set_title(f"Pred {np.argmax(row['prediction'])}")
  
  axes[index, 1].matshow(row['shap_0'], cmap=cm)
  axes[index, 1].axis('off')

  axes[index, 2].matshow(row['shap_1'], cmap=cm)
  axes[index, 2].axis('off')

  axes[index, 3].matshow(row['occlusion_0'])
  axes[index, 3].axis('off')

  axes[index, 4].matshow(row['occlusion_1'])
  axes[index, 4].axis('off')

  # axes[2].matshow(heatmap1)
  # axes[2].set_title("Action")
  # axes[0].imshow(img_array[0])

  plt.savefig('all_maps.pdf')

In [ ]:
maps_df.to_pickle("/content/drive/MyDrive/UU/Pattern Recognition/PR project/4. Data analysis/shap_and_occlusion_maps.pickle")

# GradCam (IGNORE)

This is pretty much a "manual" implementation of GradCam rather than a package. It follows the following tutorial: https://keras.io/examples/vision/grad_cam/

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:

img_size = (224, 224)
#preprocess_input = keras.applications.xception.preprocess_input
#decode_predictions = keras.applications.xception.decode_predictions

last_conv_layer_index = -4 

# The local path to our target image
img_path = list(uploaded.keys())[0]

#display(Image(img_path))

In [ ]:


img_to_batch(img_path).shape


In [ ]:
# function (from tutorial) to create GradCAM heatmap

# \@TINGTING This is my code from my other class but there is some mistake here

def make_gradcam_heatmap(img_array, model, last_conv_layer_index, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.layers[last_conv_layer].output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        print(grad_model(img_array)[1])
        if pred_index is None:
            pred_index = np.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [ ]:
img_array = img_to_batch(img_path)
heatmap0 = make_gradcam_heatmap(img_array, model, last_conv_layer_index, pred_index = 0)
heatmap1 = make_gradcam_heatmap(img_array, model, last_conv_layer_index, pred_index = 1)


In [ ]:
# Display heatmap

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12,4))
axes[1].matshow(heatmap0)
axes[1].set_title("Not Action")
axes[2].matshow(heatmap1)
axes[2].set_title("Action")
axes[0].imshow(img_array[0])
